In [46]:
import numpy as np
import torch
from scipy.signal import welch
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


# Load models
generator = load_model("saved_models/qpo_cgan_phy_generator.keras")
posterior = torch.load("trained_sbi_posterior.pt")


# --- Lorentzian Fit Model ---
def lorentzian(f, A, f0, gamma):
    return A / (1 + ((f - f0) / gamma)**2)

# --- Safe Q Estimation ---


def compute_lorentzian_q(series, fs=1.0, f_window=(0.001, 0.5)):
    """
    Compute the Q-factor from Lorentzian fit of the PSD.

    Parameters:
    - series (array): 1D light curve (flux values)
    - fs (float): Sampling frequency (default 1.0 Hz)
    - f_window (tuple): Frequency range to search for peak

    Returns:
    - Q (float): Quality factor of Lorentzian peak (or 0.0 if fit fails)
    """
    try:
        # Compute PSD
        f, Pxx = welch(series.squeeze(), fs=fs, nperseg=256)

        # Frequency window to focus on possible QPOs
        mask = (f > f_window[0]) & (f < f_window[1])
        f_peak = f[mask]
        Pxx_peak = Pxx[mask]

        # Fit Lorentzian: A / (1 + ((f - f0) / gamma)^2)
        p0 = [np.max(Pxx_peak), f_peak[np.argmax(Pxx_peak)],
              0.01]  # [A, f0, gamma]
        popt, _ = curve_fit(lorentzian, f_peak, Pxx_peak, p0=p0, maxfev=5000)
        A, f0, gamma = popt

        if gamma <= 0 or f0 <= 0:
            print("Lorentzian fit returned non-physical gamma or f0. Setting Q = 0.")
            return 0.0

        Q = f0 / gamma
        return Q

    except Exception as e:
        print(f"Lorentzian fit failed: {e}. Setting Q = 0.")
        return 0.0


# --- QPO Detection with SBI Posterior ---
def detect_qpo_sbi(curve, posterior, fs=1.0, show_plot=False):
    f, Pxx = welch(curve.squeeze(), fs=fs, nperseg=256)
    x_obs = torch.tensor(Pxx, dtype=torch.float32)

    # Posterior sampling
    samples = posterior.sample((500,), x=x_obs, show_progress_bars=False)
    fc_samples = samples[:, 0].numpy()
    amp_samples = samples[:, 1].numpy()

    fc_mean = fc_samples.mean()
    fc_std = fc_samples.std()
    amp_mean = amp_samples.mean()

    Q = compute_lorentzian_q(curve, fs=fs)

    print(
        f"Posterior fc range: {fc_samples.min():.3f} – {fc_samples.max():.3f}")
    print(
        f"Posterior amp range: {amp_samples.min():.3f} – {amp_samples.max():.3f}")
    print(f"Q = {Q:.2f} | fc_std = {fc_std:.3f} | amp_mean = {amp_mean:.3f}")

    if show_plot:
        plt.figure(figsize=(12, 4))
        plt.semilogy(f, Pxx)
        plt.title("PSD of Input Light Curve")
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # Score-based decision
    score = Q / 3 + amp_mean - fc_std
    has_qpo = score > 0.5

    return {
        "fc_mean": fc_mean,
        "fc_std": fc_std,
        "amp_mean": amp_mean,
        "Q": Q,
        "qpo": has_qpo,
        "score": score,
        "samples": samples
    }

/var/folders/15/7vdcd9756072lbp614rqt18c0000gn/T/ipykernel_55016/47894721.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  posterior = torch.load("trained_sbi_posterior.

In [ ]:
# Load data
data = np.loadtxt("ltcrv4bands_rej_dt100.dat")
bands = [data[:, i] for i in range(4)]

print(" QPO Detection on XMM-Newton Bands (REJ1034+396)\n")

for i, band in enumerate(bands):
    print(f"Band {i+1}:")
    result = detect_qpo_sbi(band, posterior, fs=1.0, show_plot=False)
    print(
        f"→ fc_mean: {result['fc_mean']:.3f}, fc_std: {result['fc_std']:.3f}")
    print(f"→ amp_mean: {result['amp_mean']:.3f}, Q: {result['Q']:.2f}")
    print(
        f"→ QPO Detected? {'YES' if result['qpo'] else 'NO'} | Score: {result['score']:.2f}\n")

🔍 QPO Detection on XMM-Newton Bands (REJ1034+396)

Band 1:
Posterior fc range: 0.032 – 0.995
Posterior amp range: 0.101 – 0.989
Q = 2.71 | fc_std = 0.234 | amp_mean = 0.553
→ fc_mean: 0.534, fc_std: 0.234
→ amp_mean: 0.553, Q: 2.71
→ QPO Detected? YES | Score: 1.22

Band 2:
Posterior fc range: 0.020 – 0.995
Posterior amp range: 0.115 – 0.999
Q = 5.52 | fc_std = 0.227 | amp_mean = 0.546
→ fc_mean: 0.514, fc_std: 0.227
→ amp_mean: 0.546, Q: 5.52
→ QPO Detected? YES | Score: 2.16

Band 3:
Posterior fc range: 0.017 – 0.998
Posterior amp range: 0.103 – 0.999
Q = 5.39 | fc_std = 0.235 | amp_mean = 0.536
→ fc_mean: 0.512, fc_std: 0.235
→ amp_mean: 0.536, Q: 5.39
→ QPO Detected? YES | Score: 2.10

Band 4:
Lorentzian fit returned non-physical gamma or f0. Setting Q = 0.
Posterior fc range: 0.018 – 0.981
Posterior amp range: 0.101 – 0.988
Q = 0.00 | fc_std = 0.227 | amp_mean = 0.528
→ fc_mean: 0.513, fc_std: 0.227
→ amp_mean: 0.528, Q: 0.00
→ QPO Detected? NO | Score: 0.30



In [48]:
import tensorflow as tf

# Load your trained generator and posterior (if not already loaded)
generator = tf.keras.models.load_model(
    "saved_models/qpo_cgan_phy_generator.keras")
# posterior = torch.load("trained_sbi_posterior.pt")  # already loaded

latent_dim = 100
num_samples = 100
results = []

# Range of test values for frequency and amplitude
fc_range = (0.01, 1)
amp_range = (0.6, 1.0)

for i in range(num_samples):
    # Random test params
    fc = np.random.uniform(*fc_range)
    amp = np.random.uniform(*amp_range)

    # Generate QPO and non-QPO light curves
    z = tf.random.normal((1, latent_dim))

    label_qpo = tf.convert_to_tensor([[fc, amp, 1.0]], dtype=tf.float32)

    signal_qpo = generator([z, label_qpo], training=False).numpy().squeeze()

    # Detect QPO
    result_qpo = detect_qpo_sbi(signal_qpo, posterior, fs=1.0, show_plot=False)

    results.append({
        "true_qpo": 1,
        "fc": fc,
        "amp": amp,
        "detected": int(result_qpo["qpo"]),
        "score": result_qpo["score"],
        "Q": result_qpo["Q"],
        "amp_mean": result_qpo["amp_mean"],
        "fc_std": result_qpo["fc_std"]
    })

Posterior fc range: 0.020 – 0.999
Posterior amp range: 0.105 – 0.999
Q = 270.97 | fc_std = 0.219 | amp_mean = 0.551
Posterior fc range: 0.010 – 0.987
Posterior amp range: 0.102 – 0.999
Q = 1.07 | fc_std = 0.224 | amp_mean = 0.518
Posterior fc range: 0.027 – 0.999
Posterior amp range: 0.106 – 0.987
Q = 93.93 | fc_std = 0.227 | amp_mean = 0.578
Posterior fc range: 0.019 – 0.998
Posterior amp range: 0.100 – 0.977
Q = 0.85 | fc_std = 0.217 | amp_mean = 0.552
Posterior fc range: 0.012 – 0.998
Posterior amp range: 0.102 – 0.998
Q = 0.82 | fc_std = 0.230 | amp_mean = 0.536
Posterior fc range: 0.019 – 0.999
Posterior amp range: 0.103 – 0.991
Q = 73.83 | fc_std = 0.235 | amp_mean = 0.573
Lorentzian fit returned non-physical gamma or f0. Setting Q = 0.
Posterior fc range: 0.020 – 0.986
Posterior amp range: 0.114 – 0.997
Q = 0.00 | fc_std = 0.218 | amp_mean = 0.581
Posterior fc range: 0.014 – 0.998
Posterior amp range: 0.101 – 0.998
Q = 0.75 | fc_std = 0.228 | amp_mean = 0.542
Lorentzian fit retu

In [49]:
import pandas as pd
df = pd.DataFrame(results)
# ignoring the failed cases Lorentzian fit returned non-physical gamma or f0
df = df[df['Q'] != 0]
accuracy = (df['true_qpo'] == df['detected']).mean()
print(f"Detector Accuracy on GAN samples: {accuracy*100:.2f}%")

Detector Accuracy on GAN samples: 94.03%
